In [ ]:
import pandas as pd

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


import random

from collections import defaultdict

In [8]:
def replace(string):
    c = '!#$%^&*(.)[]{};:,/"<>?-`@\'~”—=_·\n+123456“7890‘’'
    for i in range(len(c)):
        string = string.replace(c[i]," ")
    return(string)
data_set = pd.read_csv('./src/train.csv')
test_set = pd.read_csv('./src/test.csv')

In [9]:
data = replace(data_set["comment_text"].str.cat(sep = "|zkr")).split("|zkr")
data_test = replace(test_set["comment_text"].str.cat(sep = "|zkr")).split("|zkr")
test_id = test_set["id"].str.cat(sep = "|").split("|")

In [10]:
sentence = list(" ".join(d.split()) for d in data)
test_sentence = list(" ".join(d.split()) for d in data_test)

In [11]:
words = set(("".join([j for i in sentence for j in i])).split())

In [12]:
word_to_ix = defaultdict(int)
for i,word in enumerate(words):
    word_to_ix[word] = i+special_token
word_to_ix["<unk>"] = 0
word_to_ix["<pad>"] = 1

In [13]:
def sentence_to_dict(sentence,max_len,word_to_ix):
    s_split = sentence.split()
    indi = [word_to_ix[word] for word in s_split]
    if len(indi) < max_len:
        indi += [word_to_ix["<pad>"]] * (max_len - len(indi))
    else:
        indi = indi[:max_len]
    return indi

In [14]:
def create_batch(sentences,max_len,word_to_ix,batch_size,data_set):
    correct = []
    sentence = []
    for _ in range(batch_size):
        index = random.randint(0,len(sentences)-1)
        sentence.append(sentences[index])
        correct.append([data_set["toxic"][index],data_set["severe_toxic"][index],data_set["obscene"][index],data_set["threat"][index],data_set["insult"][index],\
                        data_set["identity_hate"][index]])
    sen_idx = [sentence_to_dict(sen,max_len,word_to_ix) for sen in sentence]
    return sen_idx,correct

In [15]:
def validate_batch(test_sentence,max_len,word_to_ix,batch_size,test_id,test_index):
    test_id_batch = []
    test_sentence_batch = []
    end = test_index+batch_size if test_index+batch_size < len (test_id) else len (test_id)
    for i in range(test_index,end):
        test_id_batch.append(test_id[i])
        test_sentence_batch.append(test_sentence[i])
    test_sen_idx = [sentence_to_dict(sen,max_len,word_to_ix) for sen in test_sentence_batch]
    return test_sen_idx,test_id_batch